In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competicao-um-ic/train.csv
/kaggle/input/competicao-um-ic/test.csv


In [6]:
from sklearn import tree
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

treino = pd.read_csv('/kaggle/input/competicao-um-ic/train.csv')
teste = pd.read_csv('/kaggle/input/competicao-um-ic/test.csv')

def preTratamento(df): #define valores para campos nulos

    df.DS_UNIDADE_TEMPO_DOENCA = df.DS_UNIDADE_TEMPO_DOENCA.fillna('0')
    df.DS_INDICACAO_ACIDENTE = df.DS_INDICACAO_ACIDENTE.fillna('0')
    df.DS_TIPO_ATENDIMENTO = df.DS_TIPO_ATENDIMENTO.fillna('0')
    df.DS_TIPO_INTERNACAO = df.DS_TIPO_INTERNACAO.fillna('0')
    df.DS_CARATER_ATENDIMENTO = df.DS_CARATER_ATENDIMENTO.fillna('0')
    df.DS_TIPO_PREST_SOLICITANTE = df.DS_TIPO_PREST_SOLICITANTE.fillna('0')
    df.DS_TIPO_GUIA = df.DS_TIPO_GUIA.fillna('0')
    df.DS_GRUPO = df.DS_GRUPO.fillna('0')
    df.DS_CBO = df.DS_CBO.fillna('0')
    df.DS_SUBGRUPO = df.DS_SUBGRUPO.fillna('0')

    df.QT_TEMPO_DOENCA = df.QT_TEMPO_DOENCA.fillna(0)
    df.QT_DIA_SOLICITADO = df.QT_DIA_SOLICITADO.fillna(0)
    df.CD_GUIA_REFERENCIA = df.CD_GUIA_REFERENCIA.fillna(0)
    df.QT_SOLICITADA = df.QT_SOLICITADA.fillna(0)
    df.CD_ITEM = df.CD_ITEM.fillna(0)
    return df

treinamento = preTratamento(treino)
numerico = treinamento[['QT_TEMPO_DOENCA', 'QT_SOLICITADA',  'QT_DIA_SOLICITADO', 'CD_GUIA_REFERENCIA']]
categorico = treinamento[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE','DS_SUBGRUPO', 'DS_TIPO_GUIA']]
classe = treinamento['DS_STATUS_ITEM']

x = pd.concat([numerico, categorico], axis=1) #concatena em uma só tabela os valores númericos e categoricos pré-tratados
x = x.dropna()
y = classe

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=0)

def standardScalerFunc(data):
  ss = StandardScaler()
  sstransformed = ss.fit_transform(data)

  return ss, pd.DataFrame(sstransformed)

def oneHotEncoderFunc(data):
  x2 = OneHotEncoder(dtype='float',sparse=False, handle_unknown='ignore')
  ohetransformed = x2.fit_transform(data)
  
  return x2, pd.DataFrame(ohetransformed)

parameters = tree.DecisionTreeClassifier(max_depth=None, criterion='gini')
#parameters = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='crammer_singer', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=10, max_iter=1000)
#parameters = svm.SVC(kernel ='poly', max_iter = 1000)

x1, y1 = standardScalerFunc(X_train[['QT_TEMPO_DOENCA', 'QT_SOLICITADA',  'QT_DIA_SOLICITADO', 'CD_GUIA_REFERENCIA']])
x2, y2 = oneHotEncoderFunc(X_train[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE','DS_SUBGRUPO', 'DS_TIPO_GUIA']])

x_test_ss = x1.transform(X_test[['QT_TEMPO_DOENCA', 'QT_SOLICITADA',  'QT_DIA_SOLICITADO', 'CD_GUIA_REFERENCIA']])
x_test_ohe = x2.transform(X_test[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE','DS_SUBGRUPO', 'DS_TIPO_GUIA']])

Btest = pd.concat([pd.DataFrame(x_test_ss), pd.DataFrame(x_test_ohe)], axis=1)
Btrain = pd.concat([y1, y2], axis=1)
parameters.fit(Btrain, y_train)
y_pred = parameters.predict(Btest)
accuracy = accuracy_score(y_test, parameters.predict(Btest))
confusion = confusion_matrix(y_test,y_pred)
report = classification_report(y_test,y_pred)
print("Report: \n",report)
print("Matriz de confusão: \n",confusion)
print("Acurácia: \n",accuracy)
#no_dados = tree.export_graphviz(parameters, out_file=None, max_depth=None, feature_names=None, class_names=treino.target_names, label='all', filled=False, leaves_parallel=False, impurity=True, node_ids=False, proportion=False, rotate=False, rounded=False, special_characters=False, precision=3, fontname='helvetica')

In [ ]:
from itertools import zip_longest
teste1 = preTratamento(teste)

testx1 = x1.transform(teste1[['QT_TEMPO_DOENCA', 'QT_SOLICITADA',  'QT_DIA_SOLICITADO', 'CD_GUIA_REFERENCIA']])
testx2 = x2.transform(teste1[['DS_UNIDADE_TEMPO_DOENCA', 'DS_TIPO_INTERNACAO', 'DS_CARATER_ATENDIMENTO', 'DS_TIPO_ATENDIMENTO', 'DS_INDICACAO_ACIDENTE', 'DS_TIPO_PREST_SOLICITANTE','DS_SUBGRUPO', 'DS_TIPO_GUIA']])

test = pd.concat([pd.DataFrame(testx1), pd.DataFrame(testx2)], axis=1)

tab = parameters.predict(test)
submission = pd.DataFrame.from_records(zip_longest(teste['Unnamed: 0'], tab), columns=['ID', 'DS_STATUS_ITEM'])
submission.set_index('ID').to_csv('submission.csv')